## Manos a la obra con Pinecone (VDB)

In [ ]:
! pip install "pinecone[grpc]"

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv 
from pinecone import Pinecone, ServerlessSpec

load_dotenv(find_dotenv())

pinecone_api_key = os.getenv("PINECONE_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)


In [26]:
from langchain_openai import OpenAIEmbeddings

embeddings_model = OpenAIEmbeddings(model="text-embedding-3-large")

In [27]:
def get_embedding(text_to_embed):
    text = text_to_embed.replace("\n", " ")
    return embeddings_model.embed_documents([text])

In [ ]:
# Ejemplo de datos para crear embeddings
data = [
    {"id": "vec1", "text": "Orange es una fruta cítrica conocida por su jugo refrescante y su sabor ácido-dulce."},
    {"id": "vec2", "text": "La empresa de telecomunicaciones Orange ofrece servicios de telefonía móvil e Internet en muchos países."},
    {"id": "vec3", "text": "Las naranjas son una excelente fuente de vitamina C y se consumen en todo el mundo."},
    {"id": "vec4", "text": "Orange ha sido pionera en la industria de las telecomunicaciones, proporcionando cobertura de red y servicios avanzados."},
    {"id": "vec5", "text": "Tomar un vaso de jugo de naranja fresco es una forma popular de comenzar el día."},
    {"id": "vec6", "text": "La compañía Orange fue fundada para proporcionar servicios de telecomunicaciones modernos y eficientes."}
]

# Genera embeddings usando un modelo de OpenAI
embeddings = [get_embedding(item["text"]) for item in data]

In [34]:
index_name = "lil-demo-index"

spec = ServerlessSpec(
    cloud="aws",
    region="us-east-1",
)

# Crea un índice si no existe
if index_name not in pc.list_indexes():
    pc.create_index(index_name, dimension=len(embeddings[0][0]), spec=spec , metric="cosine")

# Conecta al índice creado
index = pc.Index(index_name)


In [35]:
# Prepara los registros para insertar
records = []
for d, e in zip(data, embeddings):
    records.append({
        "id": d['id'],
        "values": e[0],
        "metadata": {'text': d['text']}
    })

# Inserta los vectores en el índice
index.upsert(vectors=records)


{'upserted_count': 6}

In [36]:
# Convierte la consulta a un vector de embedding
consulta = "Tell me about the company Orange"
consulta_embedding = embeddings_model.embed_query(consulta)

# Realiza la búsqueda en el índice
resultado = index.query(top_k=3, vector=consulta_embedding, include_metadata=True)

# Muestra los resultados
print(resultado)


{'matches': [{'id': 'vec4',
              'metadata': {'text': 'Orange ha sido pionera en la industria de '
                                   'las telecomunicaciones, proporcionando '
                                   'cobertura de red y servicios avanzados.'},
              'score': 0.521476448,
              'values': []},
             {'id': 'vec2',
              'metadata': {'text': 'La empresa de telecomunicaciones Orange '
                                   'ofrece servicios de telefonía móvil e '
                                   'Internet en muchos países.'},
              'score': 0.501588643,
              'values': []},
             {'id': 'vec6',
              'metadata': {'text': 'La compañía Orange fue fundada para '
                                   'proporcionar servicios de '
                                   'telecomunicaciones modernos y eficientes.'},
              'score': 0.500831783,
              'values': []}],
 'namespace': '',
 'usage': {'read_units': 6}

In [33]:
# Elimina el índice de Pinecone
pc.delete_index(index_name)
